# 1. Time series gene regulatory network inference
We want to infer the gene regulatory network (denote with adjacency matrix $\mathbf{A}$) given the time-series gene expression data $x$. The gene expression data records the change of $10$ gene expression values from time point $1$ to $50$. 

In the lecture, we talked about multiple ways to infer the gene regulatory network with data driven methods, one of the most straightforward ways is to use the differential equation model. Let's assume the gene expression data follows:

$$
\frac{d\mathbf{x}(t)}{dt} = \mathbf{Ax}(t) + \mathbf{n},\, \mathbf{n}\sim \mathcal{N}(\mathbf{0},\mathbf{I})
$$

We can estimate the changing speed of gene expression data $\frac{d\mathbf{x}(t)}{dt}$ at time point $t$($t = 1,2,3,\cdots,49$) with $\Delta \mathbf{x}(t) = \mathbf{x}(t+1) - \mathbf{x}(t)$. And the problem turns into a [least square](https://en.wikipedia.org/wiki/Least_squares#Lasso_method) problem

$$
\mathbf{A} = \arg\min_{\mathbf{A}}\sum_{t = 1}^{49}\Vert\Delta \mathbf{x}(t) - \mathbf{Ax}(t)\Vert_2^2 
$$

Please find the solution of the least square problem above using python. 

(Hint: You need to
* Calculate $\Delta \mathbf{x}(t)$ using $\mathbf{x}(t)$
* Calculate optimal $\mathbf{A}$ using [gradient descent algorithm](https://en.wikipedia.org/wiki/Gradient_descent#:~:text=Gradient%20descent%20is%20a%20first,the%20direction%20of%20steepest%20descent), or by setting the gradient of the objective function(w.r.t $\mathbf{A}$) to be $0$. 

)


In [1]:
import numpy as np 

In [23]:
# xs stores the time series data, it is of the dimension (10,50), correspond to the expression data of 10 genes across 50 time points
xs = np.load("counts.npy")
# TODO: implement your code here

# delt_x
xs_del_0 = np.delete(xs, 0, 1)
xs_del_49 = np.delete(xs, 49, 1)
delt_x = np.subtract(xs_del_0, xs_del_49)


def gradient(A_cal):
    return -2 * np.dot(delt_x - np.dot(A_cal,xs_del_49), xs_del_49.T) 


def gradient_descent(
    gradient, start, learn_rate=0.0001, n_iter=200, tolerance=1e-06
):
    A_cal = start
    A_history = A_cal
    f_history = delt_x - np.dot(A_cal,xs_del_49)
    for _ in range(n_iter):
        diff = -learn_rate * gradient(A_cal)
        if np.all(np.abs(diff) <= tolerance):
            break
        A_cal += diff
        A_history = np.vstack((A_history,A_cal))
        f_history = np.vstack((f_history,delt_x - np.dot(A_cal,xs_del_49)))
    return A_cal, A_history, f_history

A1, A1_history, f1_history = gradient_descent(gradient,np.zeros((10,10)))
A1

array([[1.54244002e-04, 1.46574887e-05, 2.66614023e-04, 9.05815137e-05,
        2.13589287e-04, 7.04775028e-05, 2.65738754e-04, 1.32969240e-04,
        9.96570725e-05, 2.16704979e-04],
       [1.25119379e-04, 1.20106117e-05, 2.16365424e-04, 7.33864786e-05,
        1.73078891e-04, 5.73989698e-05, 2.15531030e-04, 1.07841564e-04,
        8.08221837e-05, 1.75829628e-04],
       [3.52910619e-04, 3.40029031e-05, 6.10376366e-04, 2.06898157e-04,
        4.87997133e-04, 1.62137899e-04, 6.07893236e-04, 3.04155943e-04,
        2.27948181e-04, 4.95988640e-04],
       [7.30606801e-06, 8.04356101e-07, 1.27143140e-05, 4.20738904e-06,
        9.95294508e-06, 3.54716372e-06, 1.25594819e-05, 6.27995005e-06,
        4.70454364e-06, 1.03040954e-05],
       [4.91025546e-05, 4.70454364e-06, 8.49047163e-05, 2.88069944e-05,
        6.79374434e-05, 2.25089745e-05, 8.45865013e-05, 4.23234583e-05,
        3.17196291e-05, 6.90003776e-05],
       [2.80301706e-04, 2.71046882e-05, 4.84871563e-04, 1.64256479e-04,
   

# 2. Incorporate l1 term 


Gene regulatory network tend to be a sparse graph(have a very small number of $1$s), which can be learned with an $\ell 1$ regularization term. By adding the $\ell 1$ regularization term into the objective function above, the problem turns into a LASSO regression problem:

$$
\mathbf{A} = \arg\min_{\mathbf{A}}\sum_{t = 1}^{49}\Vert\Delta \mathbf{x}(t) - \mathbf{Ax}(t)\Vert_2^2 + \lambda * \Vert\mathbf{A}\Vert_1
$$

$$
\Vert\mathbf{A}\Vert_1 = \sum_i\sum_j| A_{ij}|_1
$$

It is hard to solve a lasso regression problem using traditional gradient descent algorithm, since $\ell 1$ regularization is non-differentiable. **Iterative soft thresholding algorithm** (**ISTA**, a proximal gradient descent algorithm) is a good choice for solving such problem. Please implement ISTA algorithm in the block below.

* Information about ISTA algorithm: https://www.stat.cmu.edu/~ryantibs/convexopt-F15/lectures/08-prox-grad.pdf


In [39]:
xs = np.load("counts.npy")
# TODO: implement your code here

def  soft_thresholding_operator(A_ij, Lamb):
  if A_ij > Lamb:
    return A_ij - Lamb
  elif A_ij < -Lamb:
    return A_ij + Lamb
  else:
    return 0

vfunc = np.vectorize(soft_thresholding_operator)

def ISTA(start, learn_rate=0.0001, n_iter=300, Lamb = 2.8e-6 #, tolerance=1e-05
         ):
    A_cal = start
   # A_history = A_cal
    #f_history = delt_x - np.dot(A_cal,xs_del_49)
    for _ in range(n_iter):
        A_cal = A_cal + learn_rate * np.dot(delt_x - np.dot(A_cal,xs_del_49), xs_del_49.T) 
        #for each element in A_cal, apply soft-thresholding operator here
        A_cal = vfunc(A_cal, Lamb)
        #if np.all(np.abs(delt_x - np.dot(A_cal,xs_del_49)) <= tolerance):
        #    break
        #A_history = np.vstack((A_history,A_cal))
        #f_history = np.vstack((f_history,delt_x - np.dot(A_cal,xs_del_49)))
    return A_cal #, A_history, f_history


A2= ISTA(np.zeros((10,10)))
A2

array([[5.30728736e-09, 0.00000000e+00, 1.22576590e-05, 0.00000000e+00,
        6.45325315e-06, 0.00000000e+00, 1.21499415e-05, 0.00000000e+00,
        0.00000000e+00, 6.81489963e-06],
       [0.00000000e+00, 0.00000000e+00, 6.89173333e-06, 0.00000000e+00,
        2.15034774e-06, 0.00000000e+00, 6.79960717e-06, 0.00000000e+00,
        0.00000000e+00, 2.45291335e-06],
       [2.10347312e-05, 0.00000000e+00, 4.86315148e-05, 5.38643681e-06,
        3.55276286e-05, 5.68957691e-07, 4.83726161e-05, 1.58090412e-05,
        7.63882211e-06, 3.63712500e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [1.33615132e-05, 0.00000000e+00, 3.53640316e-05, 8.84296322e-07,
   

# 3. Benchmark
Calculate the difference between the ground truth gene regulatory network(denote as `A`) and the inferred gene regulatory network (step 1 result denoted as `A1`, step 2 result denoted as `A2`) using the `diff` function below. Adjust the hyper-parameter within the algorithms in step $1$ and $2$, and report the best result you can obtain(The smaller the difference, the better the result is).

In [115]:
A = np.load("gt_grn.npy")

def diff(A_inf, A):
    A_inf = (A_inf > 2e-05).astype(np.int)
    return np.sum(np.abs(A_inf - A))/(A.shape[0] * A.shape[1])
    

In [116]:
# step1
diff(A1, A)

0.64

In [117]:
# step2
diff(A2, A)

0.28